## Auditing and Exporting Detections

In [44]:
import os
import argparse
import numpy as np
import json
import csv
import rasterio

# shapely.geometry import mapping, Polygon
# import fiona # only required for exporting to shapefiles

In [45]:
# ingest the image
infile = "data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1.tif"

img_dir = infile.split(".")[0]
prj_name = img_dir.split("/")[-1]

In [46]:
# open the output from seal_detection

with open('data/new_detections.json') as f:
    detected_labels = json.load(f)

In [5]:
# open the previous training data

with open(img_dir + '/via_region_data.json') as f:
    existing_labels = json.load(f)

In [49]:
len(detected_labels)

151

### Exporting to Shapefile

In [62]:
image_annotations = []
for key, value in detected_labels.items():
    #print(key)
    annotation = [[key][0].split("/")[-1]]
    detections = []
    for item in value:
        box = item['box']
        detections.append(box)
        print(item)
    annotation.append(detections)
    image_annotations.append(annotation)

{'box': [323, 650, 391, 707], 'score': 0.9998805522918701, 'label': 0}
{'box': [607, 18, 667, 90], 'score': 0.9995396137237549, 'label': 0}
{'box': [848, 236, 902, 313], 'score': 0.9994469881057739, 'label': 0}
{'box': [450, 569, 517, 634], 'score': 0.9992052316665649, 'label': 0}
{'box': [741, 585, 784, 627], 'score': 0.996834397315979, 'label': 0}
{'box': [884, 875, 926, 913], 'score': 0.9922680854797363, 'label': 0}
{'box': [896, 408, 938, 446], 'score': 0.9881052374839783, 'label': 0}
{'box': [452, 629, 493, 721], 'score': 0.9880115985870361, 'label': 0}
{'box': [834, 919, 871, 967], 'score': 0.9783871173858643, 'label': 0}
{'box': [93, 341, 128, 389], 'score': 0.9743793606758118, 'label': 0}
{'box': [491, 551, 532, 592], 'score': 0.9688410758972168, 'label': 0}
{'box': [17, 297, 75, 358], 'score': 0.9476227760314941, 'label': 0}
{'box': [841, 440, 877, 476], 'score': 0.9288822412490845, 'label': 0}
{'box': [884, 912, 929, 946], 'score': 0.8930488228797913, 'label': 0}
{'box': [825

{'box': [428, 282, 470, 317], 'score': 0.9189882278442383, 'label': 0}
{'box': [683, 98, 730, 134], 'score': 0.9166913032531738, 'label': 0}
{'box': [823, 62, 856, 107], 'score': 0.9129351377487183, 'label': 0}
{'box': [743, 55, 790, 89], 'score': 0.892654538154602, 'label': 0}
{'box': [678, 751, 718, 790], 'score': 0.8876728415489197, 'label': 0}
{'box': [851, 50, 898, 87], 'score': 0.884394645690918, 'label': 0}
{'box': [480, 304, 526, 333], 'score': 0.8548424243927002, 'label': 0}
{'box': [577, 161, 623, 202], 'score': 0.843210756778717, 'label': 0}
{'box': [810, 64, 848, 103], 'score': 0.7732769250869751, 'label': 0}
{'box': [462, 265, 500, 296], 'score': 0.7493804097175598, 'label': 0}
{'box': [727, 80, 766, 123], 'score': 0.7409051060676575, 'label': 0}
{'box': [789, 40, 834, 75], 'score': 0.7130497694015503, 'label': 0}
{'box': [536, 162, 589, 206], 'score': 0.5786300301551819, 'label': 0}
{'box': [495, 172, 558, 224], 'score': 0.9997248649597168, 'label': 0}
{'box': [278, 180, 

In [55]:
# ingest back in the coordinates of detections within an image referenced by their filename

with open('data/data.json') as f:
    img_data = json.load(f)

# open the satellite image
dataset = rasterio.open(infile)
    
geolocated_annotations = []
for annotation in image_annotations:
    for detection in annotation[1]:
        try:            
            local_bounding_box = np.array([[detection[0], detection[1]], [detection[2], detection[1]], [detection[2], detection[3]], [detection[0], detection[3]]]).astype(int)
            image_located_bb = local_bounding_box + [img_data["image_locations"][annotation[0]]]

            geolocated_bb = []
            for point in image_located_bb:
                geolocated_bb.append(dataset.transform * point)
            geolocated_annotations.append(geolocated_bb)
        except ValueError: # if the image doesn't have a detection
            pass

In [57]:
len(geolocated_annotations)

3005

In [61]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str')
  ])
}

with fiona.open(
    'data/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(geolocated_annotations):
        record = {
            'geometry': {'coordinates': [polygon], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', infile),
                                       ('Detection', 'Seal')
                                       ]),
            'type': 'Feature'}
        c.write(record)
        

### RetinaNet to Existing VIA

In [6]:
# add the new detections to the old via_region_data.json file

#"11_fiX1mEhK","[""2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png""]",0,"[]","[2,509.275,929.174,89.376,48.904]","{}"

for filepath, detections in detected_labels.items():
    fn = filepath.split("/")[-1]
    # TODO is deep copy correct?
    for filename_size, metadata in existing_labels.items():
        if fn == metadata["filename"]:
            for detection in detections:
                # 'box' : [x1, y1, x2, y2]
                x1 = detection["box"][0]
                y1 = detection["box"][1]
                x2 = detection["box"][2]
                y2 = detection["box"][3]
                #print(x1,x2,y1,y2)
                metadata["regions"].append({'shape_attributes': {'name': 'rect', 'x': x1, 'y': y1, 'width': x2-x1, 'height': y2-y1}, 'region_attributes': {}})
    

In [7]:
# write out new VIA file with additional detections

with open(img_dir + '/via_region_data_detections.json', 'w') as fp:
    json.dump(existing_labels, fp)